In [10]:
import pandas as pd
import re
import string
import nltk
import gensim

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from underthesea import word_tokenize

### 1.	Biểu diễn văn bản sử dụng Bag-of-Words

In [31]:
documents = [
    "Tôi thích học máy, Học máy rất thú vị, Tôi rất thích sự thú vị"]

#Khởi tạo Vectorizer
count_vec = CountVectorizer()
count_occurs = count_vec.fit_transform(documents)

In [32]:
count_occur_df = pd.DataFrame(
    (count, word) for word, count in
    zip(count_occurs.toarray().tolist()[0],
        count_vec.get_feature_names_out()))
count_occur_df.columns = ['Từ', 'Số lần']
count_occur_df.sort_values('Số lần', ascending=False, inplace=True)
count_occur_df.head()

,Từ,Số lần
0,học,2
1,máy,2
2,rất,2
4,thích,2
5,thú,2


In [19]:
print(count_vec.get_feature_names_out())

['học' 'máy' 'rất' 'sự' 'thích' 'thú' 'tôi' 'vị']


In [33]:
print(count_occurs.toarray())

[[2 2 2 1 2 2 2 2]]


In [52]:
docs = [
    "Tôi thích học máy", 
    "Học máy rất thú vị", 
    "Tôi rất thích sự thú vị"]
vectorizer2 = CountVectorizer(analyzer='word', ngram_range = (2, 2))
X2= vectorizer2.fit_transform(docs)
print(vectorizer2.get_feature_names_out())
print(X2.toarray())

['học máy' 'máy rất' 'rất thích' 'rất thú' 'sự thú' 'thích học' 'thích sự'
 'thú vị' 'tôi rất' 'tôi thích']
[[1 0 0 0 0 1 0 0 0 1]
 [1 1 0 1 0 0 0 1 0 0]
 [0 0 1 0 1 0 1 1 1 0]]


### 2.	Sử dụng TF-IDF để biểu diễn văn bản

In [147]:
from underthesea import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import math

In [139]:
documentA = 'Tôi cảm thấy rất hạnh phúc và tràn đầy năng lượng hôm nay.'
documentB = 'Hôm nay là một ngày thật tồi tệ, tôi rất buồn và thất vọng.'

In [140]:
bagOfWordsA = re.sub(f"[{re.escape(string.punctuation)}]", "", documentA).lower().split(' ')
bagOfWordsB = re.sub(f"[{re.escape(string.punctuation)}]", "", documentB).lower().split(' ')

In [141]:
uniqueWords = set(bagOfWordsA).union(set(bagOfWordsB))
uniqueWords

{'buồn',
 'cảm',
 'hôm',
 'hạnh',
 'là',
 'lượng',
 'một',
 'nay',
 'ngày',
 'năng',
 'phúc',
 'rất',
 'thất',
 'thấy',
 'thật',
 'tràn',
 'tôi',
 'tệ',
 'tồi',
 'và',
 'vọng',
 'đầy'}

In [142]:
numOfWordsA = dict.fromkeys(uniqueWords, 0)
for wordA in bagOfWordsA:
    numOfWordsA[wordA] += 1
numOfWordsB = dict.fromkeys(uniqueWords, 0)
for wordB in bagOfWordsB:
    numOfWordsB[wordB] += 1

In [143]:
numOfWordsA

{'và': 1,
 'tràn': 1,
 'buồn': 0,
 'phúc': 1,
 'nay': 1,
 'là': 0,
 'thất': 0,
 'đầy': 1,
 'vọng': 0,
 'hạnh': 1,
 'lượng': 1,
 'năng': 1,
 'thật': 0,
 'cảm': 1,
 'ngày': 0,
 'tồi': 0,
 'thấy': 1,
 'tệ': 0,
 'tôi': 1,
 'hôm': 1,
 'rất': 1,
 'một': 0}

In [144]:
def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [145]:
tfA = computeTF(numOfWordsA, bagOfWordsA)
tfB = computeTF(numOfWordsB, bagOfWordsB)

In [146]:
tfA

{'và': 0.07692307692307693,
 'tràn': 0.07692307692307693,
 'buồn': 0.0,
 'phúc': 0.07692307692307693,
 'nay': 0.07692307692307693,
 'là': 0.0,
 'thất': 0.0,
 'đầy': 0.07692307692307693,
 'vọng': 0.0,
 'hạnh': 0.07692307692307693,
 'lượng': 0.07692307692307693,
 'năng': 0.07692307692307693,
 'thật': 0.0,
 'cảm': 0.07692307692307693,
 'ngày': 0.0,
 'tồi': 0.0,
 'thấy': 0.07692307692307693,
 'tệ': 0.0,
 'tôi': 0.07692307692307693,
 'hôm': 0.07692307692307693,
 'rất': 0.07692307692307693,
 'một': 0.0}

In [160]:
def computeIDF(document):
    N = len(document)

    idfDict = dict.fromkeys(document[0].keys(), 0)
    for doc in document:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1

    for word, val in idfDict.items():
        idfDict[word] = math.log(N /float(val))
        idfDict[word] = N / float(val)
    return idfDict

In [161]:
documents_example = [numOfWordsA, numOfWordsB]
idfs = computeIDF(documents_example)

In [162]:
idfs

{'và': 1.0,
 'tràn': 2.0,
 'buồn': 2.0,
 'phúc': 2.0,
 'nay': 1.0,
 'là': 2.0,
 'thất': 2.0,
 'đầy': 2.0,
 'vọng': 2.0,
 'hạnh': 2.0,
 'lượng': 2.0,
 'năng': 2.0,
 'thật': 2.0,
 'cảm': 2.0,
 'ngày': 2.0,
 'tồi': 2.0,
 'thấy': 2.0,
 'tệ': 2.0,
 'tôi': 1.0,
 'hôm': 1.0,
 'rất': 1.0,
 'một': 2.0}

In [164]:
def computeTFIDF(tfBagOfWords, idfs):
    tfidf ={}
    for word, tf in tfBagOfWords.items():
        tfidf[word] = tf * idfs[word]
    return tfidf

In [165]:
tfidfA = computeTFIDF(tfA, idfs)
tfidfB = computeTFIDF(tfB, idfs)
df = pd.DataFrame([tfidfA, tfidfB])

In [166]:
df

,và,tràn,buồn,phúc,nay,là,thất,đầy,vọng,hạnh,...,thật,cảm,ngày,tồi,thấy,tệ,tôi,hôm,rất,một
0,0.076923,0.153846,0.000000,0.153846,0.076923,0.000000,0.000000,0.153846,0.000000,0.153846,...,0.000000,0.153846,0.000000,0.000000,0.153846,0.000000,0.076923,0.076923,0.076923,0.000000
1,0.071429,0.000000,0.142857,0.000000,0.071429,0.142857,0.142857,0.000000,0.142857,0.000000,...,0.142857,0.000000,0.142857,0.142857,0.000000,0.142857,0.071429,0.071429,0.071429,0.142857


In [173]:
tfidfDocuments = [documentA, documentB]
tokenized_docs = [word_tokenize(doc, format="text") for doc in tfidfDocuments]
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(tokenized_docs)

In [175]:
df_tfidf = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names_out())
df_tfidf

,buồn,cảm_thấy,hôm_nay,hạnh_phúc,là,một,ngày,năng_lượng,rất,thất_vọng,thật,tràn_đầy,tôi,tồi_tệ,và
0,0.000000,0.407401,0.289869,0.407401,0.000000,0.000000,0.000000,0.407401,0.289869,0.000000,0.000000,0.407401,0.289869,0.000000,0.289869
1,0.332872,0.000000,0.236841,0.000000,0.332872,0.332872,0.332872,0.000000,0.236841,0.332872,0.332872,0.000000,0.236841,0.332872,0.236841


### 3.	Thực hành với Word Embeddings (Word2Vec, GloVe, FastText)

#### Word2Vec

In [11]:
documents_word2Vec = [
    "Tôi cảm thấy rất hạnh phúc và tràn đầy năng lượng hôm nay.",
    "Hôm nay là một ngày thật tồi tệ, tôi rất buồn và thất vọng.",
    "Anh ta bình thản trước mọi khó khăn.",
    "Cô ấy sợ hãi khi ở một mình trong bóng tối."
]

In [12]:
tokenized_doc = [word_tokenize(doc, format="text").split() for doc in documents_word2Vec]

In [14]:
model = gensim.models.Word2Vec(
    sentences = tokenized_doc, # dữ liệu đầu vào
    vector_size = 50, # default value of vector_size = 100, embedding size
    window = 10,
    min_count = 1, # default value of min_count = 5
    workers = 4) # default value of workers = 3